In [ ]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd 

In [ ]:
engine = create_engine(connstr)
PK_FORM = '2021-Q4-BOD2201'
Acronym='GSA'
def get_df(query):
    conn = engine.connect() 
    df = pd.read_sql(query,con=conn) 
    conn.close()
    return df

In [23]:
df_vwQuestions = get_df(f" SELECT TOP 5000 * FROM vwQuestions WHERE PK_FORM LIKE '%{PK_FORM}%' ")
df_vwOrgSubToComponent = get_df(f" SELECT TOP 5000 * FROM  vwOrgSubToComponent WHERE PK_FORM LIKE '%{PK_FORM}%' AND Acronym = '{Acronym}' ORDER BY PK_OrgSubmission DESC" )
df_vwMetricsAnswers = get_df(f"SELECT TOP 5000 * FROM  vwMetricsAnswers WHERE PK_FORM LIKE '%{PK_FORM}%' AND Acronym = '{Acronym}' ORDER BY PK_Question DESC" ) 
df_Agencies = get_df(f" SELECT DISTINCT Acronym, PK_Component, Component FROM vwOrgSubToComponent WHERE PK_ReportingCycle = (SELECT MAX(PK_ReportingCycle) FROM fsma_ReportingCycles) " ) 

In [43]:
agencies = ['GSA', 'DOJ', 'DHS', 'SBA', 'EPA', 'NASA']
df = get_df(" SELECT * FROM CVEs ") 
df["Agency"] = agencies[0]
df["CVE ID"] = df['PK_CISA_CVE'] 
df["Count"] = df["CVE ID"].apply(lambda s:  re.sub(r"CVE\-\d{4}\-(\d{2})(\d{1,4})", r"\2", s))
df = df[['Agency', 'CVE ID','Count']] 
dff = pd.DataFrame()
for a in agencies:
    df = df_CVEs
    df["Agency"] = a
    dff = pd.concat([dff, df])
dff
dff.to_csv(r'C:\Users\Tim\Downloads\BOD_2201_CDMimport_VALID.csv', index=False)

In [ ]:
df = pd.read_excel(r'C:\Users\Tim\Downloads\BOD2201_2021_2_VALID.xlsx')
df.fillna('')
df.loc[0:3, ['Unremediated Count Justification']] = ''
df.loc[0:3, ['Challenges and Constraints']] = ' invalid attempt'
df.loc[0:3, ['Total Unremediated Assets']] = '110011' 
df.to_excel(r'C:\Users\Tim\Downloads\BOD2201_2021_2_INVALID.xlsx', index=False) 

In [ ]:
query = f""" \
    SELECT TOP 5000 * FROM BOD_CDM_IMPORT 
"""  
df_BOD_CDM_IMPORT = get_df(query)
df_BOD_CDM_IMPORT

In [ ]:
query = """ \
SELECT TOP 5000 
* 
FROM CVEUnremediated
WHERE PK_OrgSubmission = 29150 
"""   
df_CVEUN = get_df(query)
 

In [ ]:
cols = ', '.join(df_CVEUN.columns[1:])
cols

In [ ]:
cols = 'PK_OrgSubmission, PK_CISA_CVE, UnremediatedCount, Constraints, UserId, DateUploaded, DateModified, isActive, CveCountJustification'
cols

In [ ]:
vals = ''
h = 'INSERT INTO CVEUnremediated (PK_OrgSubmission, PK_CISA_CVE, UnremediatedCount, Constraints, UserId, DateUploaded, DateModified, isActive, CveCountJustification)'
h = h + '\nVALUES\n'
for i, r in df_CVEUN.iterrows(): 
    s = ''
    for c in df_CVEUN.columns[1:]: 
        s = s + f"{  r[c]  }," 
    vals = vals + f'({ s[:-1] }),\n'     
print( h ) 

vals = vals[:-1]
vals = vals.replace("None", "NULL") 
vals = re.sub(r"(CVE\-\d+\-\d+)", "'\\1'", vals)

print( vals ) 

In [ ]:
df = pd.read_csv(r'C:\Users\Tim\Downloads\BOD_CDM_Import.csv')
df

In [ ]:
dff = df.drop_duplicates(subset=['CVE ID', 'Count'], keep='last')
dff.to_csv(r'C:\Users\Tim\Downloads\BOD_CDM_Import.csv')
df = pd.read_csv(r'C:\Users\Tim\Downloads\BOD_CDM_Import.csv')
df

In [ ]:
df = pd.read_excel(r'C:\Users\Tim\Downloads\BOD2201_2021_2_VALID.xlsx').fillna('')
df['Unremediated Count Justification'] = 'Justification' 
df.columns = [c.replace(' ' , ' ') for c in df.columns] 
df.loc[df.CVE=='CVE-2011-3544', 'Unremediated Count Justification'] = ''
df.loc[df.CVE=='CVE-2011-3544', 'Total Unremediated Assets'] = '9999999'
df.columns
df.to_excel(r'C:\Users\Tim\Downloads\BOD2201_2021_2_INVALID.xlsx', index=False) 
df

df.to_sql('CVEUnremediated_UPLOAD', engine, index=True, if_exists='replace')

conn = engine.connect() 
conn.execute('DROP TABLE CVEUnremediated_UPLOAD;')
conn.close()